# Exhaustive Search : Depth First Strategy

In [1]:
# Make sure the libraries are reloaded
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from src.environment.ADR_Environment import ADR_Environment
from src.simulator.Simulator import Simulator
import yaml
import pandas as pd
import csv
from tqdm import tqdm

In [3]:
with open("src/config/exhaustive_config_10.yaml") as file: # change file name to use different sweep
        config = yaml.load(file, Loader=yaml.FullLoader)


n_debris = config['environment_parameters']['total_n_debris']

In [4]:
def generate_permutations(nums, length):
    def backtrack(visited, path):
        if len(path) == length:
            permutations.append(path[:])
            return
        for num in nums:
            if num not in visited:
                visited.add(num)
                path.append(num)
                backtrack(visited, path)
                path.pop()
                visited.remove(num)

    permutations = []
    backtrack(set(), [])
    return permutations

nums_range = range(0, n_debris)  # Range of available integers
combination_length = 6   # Length of the combination
combinations = generate_permutations(nums_range, combination_length)

for i in nums_range:
    print(i)


0
1
2
3
4
5
6
7
8
9


In [5]:
sum_rewards_per_combination = []
fuel_used = []
time_used = []

In [6]:

env = ADR_Environment()
env.env_init(env_info=config['environment_parameters'])
for i, combination in tqdm(enumerate(combinations)):
# rl episode
    # print(f"combination {i}")
    is_terminal = False
    sum_rewards = 0.0
    action_key = combination[0]
    first_debris, _ = env.action_space[action_key]
    # print('days:', _)
    observation = env.env_start(first_debris = first_debris)
    action_id = 0
    num_steps = 0
    for action in combination[1:]:
        (reward, last_state, term) = env.env_step(action)
        sum_rewards += reward
        num_steps += 1
        if term:
            break
    # print(f"Combination: {combination}, Reward: {sum_rewards}")
    sum_rewards_per_combination.append(sum_rewards)
    fuel_used.append(env.dv_max_per_mission - last_state[2])
    time_used.append(env.dt_max_per_mission - last_state[3])

df = pd.DataFrame({'combination': combinations, 'sum_rewards': sum_rewards_per_combination, 'fuel_used': fuel_used, 'time_used': time_used})

/Users/hugoderohanwillner/anaconda3/envs/Lab_Project/lib/python3.8/site-packages/astropy/units/decorators.py:316: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)


first debris:  4
debris number:  0
/n 
7226.550529766098 km 0.010801999999999999 86.4338 deg 2.351 deg 338.5518 deg 71.5766 deg
debris number:  1
/n 
7106.461483928948 km 0.0129868 86.5411 deg 4.6673 deg 91.3839 deg -89.77539999999999 deg
debris number:  2
/n 
7216.146702452183 km 0.0156286 86.4272 deg 19.7079 deg 153.7632 deg -152.8419 deg
debris number:  3
/n 
7201.846122728895 km 0.014683799999999999 86.4253 deg 11.7099 deg 78.3135 deg 45.7724 deg
debris number:  4
/n 
7100.157332934505 km 0.006514 86.5626 deg 18.899 deg 73.3074 deg -72.47320000000002 deg
debris number:  5
/n 
7184.294821499346 km 0.013622299999999999 86.4424 deg 7.6807 deg 346.4922 deg 13.2635 deg
debris number:  6
/n 
7239.3624877810125 km 0.0174354 86.3756 deg 11.8511 deg 199.7615 deg 159.6758 deg
debris number:  7
/n 
7264.133643044708 km 0.017758 86.3538 deg 12.236 deg 277.7842 deg 80.3207 deg
debris number:  8
/n 
7175.544518481246 km 0.013760399999999999 86.4815 deg 16.7194 deg 340.2707 deg 19.319 deg
debris 

0it [00:00, ?it/s]

combination 0
days: 30
first debris:  0
debris number:  0
/n 
7226.550529766098 km 0.010801999999999999 86.4338 deg 2.351 deg 338.5518 deg 71.5766 deg
debris number:  1
/n 
7106.461483928948 km 0.0129868 86.5411 deg 4.6673 deg 91.3839 deg -89.77539999999999 deg
debris number:  2
/n 
7216.146702452183 km 0.0156286 86.4272 deg 19.7079 deg 153.7632 deg -152.8419 deg
debris number:  3
/n 
7201.846122728895 km 0.014683799999999999 86.4253 deg 11.7099 deg 78.3135 deg 45.7724 deg
debris number:  4
/n 
7100.157332934505 km 0.006514 86.5626 deg 18.899 deg 73.3074 deg -72.47320000000002 deg
debris number:  5
/n 
7184.294821499346 km 0.013622299999999999 86.4424 deg 7.6807 deg 346.4922 deg 13.2635 deg
debris number:  6
/n 
7239.3624877810125 km 0.0174354 86.3756 deg 11.8511 deg 199.7615 deg 159.6758 deg
debris number:  7
/n 
7264.133643044708 km 0.017758 86.3538 deg 12.236 deg 277.7842 deg 80.3207 deg
debris number:  8
/n 
7175.544518481246 km 0.013760399999999999 86.4815 deg 16.7194 deg 340.2707

1it [00:03,  3.98s/it]/Users/hugoderohanwillner/anaconda3/envs/Lab_Project/lib/python3.8/site-packages/astropy/units/decorators.py:316: UserWarning: Wrapping true anomaly to -π <= nu < π
  return_ = wrapped_function(*func_args, **func_kwargs)
4it [00:04,  1.27it/s]

...

1199it [01:15, 16.25it/s]

combination 1195
days: 30
first debris:  0
debris number:  0
/n 
7226.550529766098 km 0.010801999999999999 86.4338 deg 2.351 deg 338.5518 deg 71.5766 deg
debris number:  1
/n 
7106.461483928948 km 0.0129868 86.5411 deg 4.6673 deg 91.3839 deg -89.77539999999999 deg
debris number:  2
/n 
7216.146702452183 km 0.0156286 86.4272 deg 19.7079 deg 153.7632 deg -152.8419 deg
debris number:  3
/n 
7201.846122728895 km 0.014683799999999999 86.4253 deg 11.7099 deg 78.3135 deg 45.7724 deg
debris number:  4
/n 
7100.157332934505 km 0.006514 86.5626 deg 18.899 deg 73.3074 deg -72.47320000000002 deg
debris number:  5
/n 
7184.294821499346 km 0.013622299999999999 86.4424 deg 7.6807 deg 346.4922 deg 13.2635 deg
debris number:  6
/n 
7239.3624877810125 km 0.0174354 86.3756 deg 11.8511 deg 199.7615 deg 159.6758 deg
debris number:  7
/n 
7264.133643044708 km 0.017758 86.3538 deg 12.236 deg 277.7842 deg 80.3207 deg
debris number:  8
/n 
7175.544518481246 km 0.013760399999999999 86.4815 deg 16.7194 deg 340.2

1203it [01:15, 16.40it/s]

combination 1199
days: 30
first debris:  0
debris number:  0
/n 
7226.550529766098 km 0.010801999999999999 86.4338 deg 2.351 deg 338.5518 deg 71.5766 deg
debris number:  1
/n 
7106.461483928948 km 0.0129868 86.5411 deg 4.6673 deg 91.3839 deg -89.77539999999999 deg
debris number:  2
/n 
7216.146702452183 km 0.0156286 86.4272 deg 19.7079 deg 153.7632 deg -152.8419 deg
debris number:  3
/n 
7201.846122728895 km 0.014683799999999999 86.4253 deg 11.7099 deg 78.3135 deg 45.7724 deg
debris number:  4
/n 
7100.157332934505 km 0.006514 86.5626 deg 18.899 deg 73.3074 deg -72.47320000000002 deg
debris number:  5
/n 
7184.294821499346 km 0.013622299999999999 86.4424 deg 7.6807 deg 346.4922 deg 13.2635 deg
debris number:  6
/n 
7239.3624877810125 km 0.0174354 86.3756 deg 11.8511 deg 199.7615 deg 159.6758 deg
debris number:  7
/n 
7264.133643044708 km 0.017758 86.3538 deg 12.236 deg 277.7842 deg 80.3207 deg
debris number:  8
/n 
7175.544518481246 km 0.013760399999999999 86.4815 deg 16.7194 deg 340.2

1205it [01:15, 16.27it/s]

In [ ]:
df = df.sort_values(by='sum_rewards', ascending=False)
df

,combination,sum_rewards,fuel_used,time_used
263,"[4, 1, 5, 3]",3.0,1.364004,0.885143
261,"[4, 1, 5, 0]",2.0,2.408115,1.532483
74,"[1, 2, 0, 5]",2.0,2.497873,1.533669
239,"[3, 5, 4, 2]",2.0,1.795712,1.140233
311,"[5, 0, 4, 3]",2.0,2.180544,1.184961
...,...,...,...,...
182,"[3, 0, 1, 5]",0.0,0.000000,0.000000
169,"[2, 5, 0, 3]",0.0,0.000000,0.000000
165,"[2, 4, 5, 0]",0.0,0.000000,0.000000
152,"[2, 3, 4, 5]",0.0,1.370315,0.291664


In [ ]:
df.to_csv('exhaustive_sweep_results.csv', index=False)

In [ ]:
df = df[df['sum_rewards'] == df['sum_rewards'].max()]
df

,combination,sum_rewards,fuel_used,time_used
263,"[4, 1, 5, 3]",3.0,1.364004,0.885143


In [ ]:
# sort df by fuel used and output the 10 first
df = df.sort_values(by='fuel_used')

In [ ]:
df.head(10)

,combination,sum_rewards,fuel_used,time_used
263,"[4, 1, 5, 3]",3.0,1.364004,0.885143


In [ ]:
import torch

numbers = [0, 3, 3, 5]
mask = [0, 1, 1, 0]

# Convert lists to PyTorch tensors
numbers_tensor = torch.tensor(numbers)
mask_tensor = torch.tensor(mask)

# Apply the mask to the numbers tensor
result = numbers_tensor * mask_tensor

print(result.tolist())  # Convert the tensor back to a list

[0, 3, 3, 0]
